Stage 1: Data collection
- Create CV file where any stock entered data is automatically parsed and saved

In [3]:
import csv
import yfinance as yf

# Step 1: Create or locate the CSV file
csv_file = "StockData.csv"

try:
    # Check if the file already exists
    with open(csv_file, "r"):
        pass
except FileNotFoundError:
    # Create a new file with header if it doesn't exist
    with open(csv_file, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Ticker Symbol", "Average volume", "t-3 vol",
                         "t-3 change", "t-2 volume",
                         "t-2 change", "t-1 volume",
                         "1-1 change", "Result change"])

# Step 2: Take input of ticker symbol from user
ticker = input("Enter the ticker symbol: ")

# Step 3: Use Yahoo Finance API to search for the ticker
try:
    stock = yf.Ticker(ticker)
    info = stock.history(period="4d")
    print(info)

    # Step 4: Return error if ticker does not exist
    if info.empty:
        print("Ticker does not exist.")
    else:
        # Step 5: Create a new row in the CSV file
        with open(csv_file, "a", newline="") as file:
            writer = csv.writer(file)
            avgVolume = info["Volume"].mean()
            # Step 6: Add information to the row
            row = [
                ticker,
                info["Volume"].mean(),  # Average volume

                info["Volume"].iloc[0]/avgVolume,  # 3 days ago's volume
                (info["Close"].iloc[0] - info["Open"].iloc[0])/(info["Open"].iloc[0])*100,  # 3 days ago's daily % change

                info["Volume"].iloc[1]/avgVolume,  # 2 days ago's volume
                (info["Close"].iloc[1] - info["Open"].iloc[1])/(info["Open"].iloc[1])*100,  # 2 days ago's daily % change

                info["Volume"].iloc[2]/avgVolume,  # 1 day ago's volume
                (info["Close"].iloc[2] - info["Open"].iloc[2])/(info["Open"].iloc[2])*100,  # 1 day ago's daily % change

               (info["Close"].iloc[3] - info["Open"].iloc[3])/(info["Open"].iloc[3])*100,  # 0 days ago daily %
            ]
            writer.writerow(row)
            print("Data saved successfully.")
except Exception as e:
    print("An error occurred:", str(e))


                 Open   High        Low      Close   Volume  Dividends  \
Date                                                                     
2023-06-20  24.410000  24.65  23.719999  24.580000  3250700          0   
2023-06-21  24.840000  25.10  24.100000  24.430000  2404700          0   
2023-06-22  24.129999  24.33  23.760000  24.320000  1889600          0   
2023-06-23  23.780001  24.15  23.680000  23.700001  2230800          0   

            Stock Splits  
Date                      
2023-06-20             0  
2023-06-21             0  
2023-06-22             0  
2023-06-23             0  
Data saved successfully.
